In [4]:
import csv
import pprint
import re

from os import listdir

import pdfplumber

HEADERS = (
    'work_order',
    'service_code',
    'service_address',
    'latitude',
    'longitude',
    'images',
    'an_action_date',
    'an_status',
    'an_action_by',
    'an_action_notes',
    'rs_activity',
    'rs_type',
    'rs_resource',
    'rs_qty',
    'rs_units',
    'rs_performed_on',
    'an_activity',
    'an_performed_on',
    'an_notes',
)

# get the file name list
pdf_files = listdir('pdf')
# for dev, just first three
# pdf_files = pdf_files[:10]
# print(pdf_files)
pdf_files = ['WO_283650.pdf']

masterlist = []

for file_name in pdf_files:
    
    with pdfplumber.open('pdf/{}'.format(file_name)) as pdf:
        
        image_count = 0
        pages = pdf.pages

        sub_masterlist = []
        for page in pages:

            lines = page.extract_text()
            image_count += len(page.images)
            
            # check for lines as not all pages have them
            # sometimes last page is just embedded images
            if lines:

                lines = enumerate(lines.split('\n'))
                iter_lines = iter(lines)

                for line_id, line in iter_lines:
                    # get Work Order ID
                    if line.startswith('Work Order ID'):
                        work_order_id_plus = line.split('Work Order ID: ')[1]
                        work_order_id = work_order_id_plus.split(' Work Order Date: ')[0]

                    # get Service Code
                    if line.startswith('Service Code'):
                        service_code, = re.findall(r'\[([^\]]*)\]', line)

                    # get Service Address
                    if line.startswith('Service Address'):
                        index, service_address = next(iter_lines)

                    # get Latitude, Longitude
                    if line.startswith('Latitude'):
                        index, lat_lng_string = next(iter_lines)
                        lat, lng = lat_lng_string.split()

            # get the Action Notes & Result Summary tables
            tables = page.extract_tables()
            print(page, len(tables))
            
            if tables:
                for table in tables:
                    # get the Action Notes rows
                    if (len(table[0]) == 4 or (len(table[0]) == 3 and table[0][0] == 'Action Date')):
                        print('ACTION NOTES:')
                        for row in table:
                            if row[0] == 'Action Date':
                                # skip first row, it's headers
                                continue
                            sub_masterlist.append([int(work_order_id), service_code, service_address.strip(), float(lat), float(lng),] + row + ["","","","","","","","","",])
                            print(row)

                    # get the Result Summary rows
                    if len(table[0]) == 6:
#                         print('RESULT SUMMARY:')
                        for row in table:
                            if row[0] == 'Activity':
                                # skip first row, it's headers ...
                                continue
                            sub_masterlist.append([int(work_order_id), service_code, service_address.strip(), float(lat), float(lng),"","","","",] + row + ["","","",] )
#                             print(row)
                      
                    # get the Activity Notes
                    if len(table[0]) == 3 and table[0][0] == 'Activity':
#                         print('ACTIVITY NOTES:')
                        for row in table:
                            if row[0] == 'Activity':
                                # skip first row, it's headers ...
                                continue
                            sub_masterlist.append([int(work_order_id), service_code, service_address.strip(), float(lat), float(lng),"","","","","","","","","","",] + row )
#                             print(row)
                        
    #             print(">>> Found {} tables in {} on {}".format(len(page.extract_tables()) , file_name, page))
    #             print(">>> BAD PDF: {}, '{}', '{}', {} {}".format(work_order_id, service_code, service_address, lat, lng), '\n\n')

        print('>>> {}, "{}", "{}", {} {}'.format(work_order_id, service_code, service_address, lat, lng))
        for row_list in sub_masterlist:
            # inserting the image_count, now that we've looped through every page
            row_list.insert(5, image_count)
            masterlist.append(row_list)

with open("working.csv", "w") as outfile:
    writer = csv.writer(outfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    writer.writerow(HEADERS)
    for item in masterlist:
        writer.writerow(item)    

<Page:1> 3
ACTION NOTES:
['08/30/19 11:04:16 AM', 'Assigned', 'Schneider, Erick', 'Work Order Created']
['09/03/19 10:06:27 AM', 'Closed', 'Stehlar, Doug', 'Camp abandoned, trash\ncleaned up x2 bags. Work\nOrder completed by Doug\nStehlar.']
['04/02/20  7:39:23 AM', 'Complete', 'Hallett, Jackie', 'Automatically reassigned to\nStehlar, Doug by the system.']
<Page:2> 0
<Page:3> 0
>>> 283650, "SWM30", "SEE PIN LOCATION", 44.051902247761 -123.124606348008
